In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import f, norm, pearsonr, ttest_rel

!pip install --upgrade gensim
import re
from itertools import chain, permutations

import gensim.downloader as api
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine, euclidean, pdist

# Load data
study_name = "Junk Fees"
specification_name = "default persona"
human_file = f"{study_name} human data values anonymized.csv"
twin_file = f"{study_name} twins data values anonymized.csv"
df_human = pd.read_csv(human_file, header=0, skiprows=[1, 2])
df_twin = pd.read_csv(twin_file, header=0, skiprows=[1, 2])


#########################
# create new relevant columns:

# human:
categories = ["Hotel", "Car", "Ticket", "Food", "Apart", "Health", "CC"]

for cat in categories:
    # find all MCQ columns for this category
    mcq_cols = [col for col in df_human.columns if col.startswith(f"{cat}.MCQ")]
    if not mcq_cols:
        raise ValueError(f"No {cat}.MCQ column found")
    mcq_col = mcq_cols[0]
    correct_col = f"{cat}.correct"
    df_human[correct_col] = np.where(
        df_human[mcq_col].isna(), np.nan, (df_human[mcq_col] == 1).astype(int)
    )
    df_twin[correct_col] = np.where(
        df_twin[mcq_col].isna(), np.nan, (df_twin[mcq_col] == 1).astype(int)
    )

    fair_cols = [col for col in df_human.columns if col.startswith(f"{cat}.Fair")]
    if not fair_cols:
        raise ValueError(f"No {cat}.Fair column found")
    fair_col = fair_cols[0]
    rating_col = f"{cat}.fairness"

    df_human[rating_col] = df_human[fair_col]
    df_twin[rating_col] = df_twin[fair_col]

correct_cols = [f"{cat}.correct" for cat in categories]
non_nan_counts = df_human[correct_cols].notna().sum(axis=1)
if not (non_nan_counts == 6).all():
    # report rows that don’t meet the expectation
    bad_rows = non_nan_counts[non_nan_counts != 6]
    raise ValueError(
        f"Expected exactly 6 non-NaN .correct columns per row, but found counts:\n{bad_rows}"
    )
non_nan_counts = df_twin[correct_cols].notna().sum(axis=1)
if not (non_nan_counts == 6).all():
    # report rows that don’t meet the expectation
    bad_rows = non_nan_counts[non_nan_counts != 6]
    raise ValueError(
        f"Expected exactly 6 non-NaN .correct columns per row, but found counts:\n{bad_rows}"
    )
fairness_cols = [f"{cat}.fairness" for cat in categories]
non_nan_counts = df_human[fairness_cols].notna().sum(axis=1)
if not (non_nan_counts == 6).all():
    bad = non_nan_counts[non_nan_counts != 6]
    raise ValueError(f"Expected exactly 6 non-NaN fairness ratings per row, but found:\n{bad}")
non_nan_counts = df_twin[fairness_cols].notna().sum(axis=1)
if not (non_nan_counts == 6).all():
    bad = non_nan_counts[non_nan_counts != 6]
    raise ValueError(f"Expected exactly 6 non-NaN fairness ratings per row, but found:\n{bad}")
df_human["percent_correct"] = df_human[correct_cols].sum(axis=1) / 6 * 100
df_human["fairness_average"] = df_human[fairness_cols].mean(axis=1)
df_twin["percent_correct"] = df_twin[correct_cols].sum(axis=1) / 6 * 100
df_twin["fairness_average"] = df_twin[fairness_cols].mean(axis=1)

reg_cols = ["Regulation", "Support.Reg"]
df_human["reg_support"] = df_human[reg_cols].mean(axis=1)
df_twin["reg_support"] = df_twin[reg_cols].mean(axis=1)

###############################


human_file = f"{study_name} human data values anonymized processed.csv"
twin_file = f"{study_name} twins data values anonymized processed.csv"
df_twin.to_csv(twin_file, index=False)
df_human.to_csv(human_file, index=False)

df_human = df_human.set_index("TWIN_ID")
df_twin = df_twin.set_index("TWIN_ID")


##################now proceed with standard meta-analysis steps
# define relevant columns:
# condition variable names:
condition_vars = [""]
# Check if we have a real condition var
if condition_vars and condition_vars[0].strip():
    cond = condition_vars[0]
    cond_h = f"{cond}_human"
    cond_t = f"{cond}_twin"
    cond_exists = True
else:
    cond_exists = False

# raw responses:
# # all the “raw” columns
# raw_vars = list(chain(correct_cols, fairness_cols, reg_cols))
# # check
# print(len(raw_vars), "columns:", raw_vars)
# raw_vars_min=list(chain([0]*7, [1]*7, [1]*2))
# raw_vars_max = list(chain([1]*7, [7]*7, [7]*2))
# #raw responses: domain=social?
# raw_vars_social=list(chain([0]*7, [1]*7, [1]*2))
# raw_vars_social_map = dict(zip(raw_vars, raw_vars_social))
# #raw responses: domain=cognitive?
# raw_vars_cognitive=list(chain([1]*7, [0]*7, [0]*2))
# raw_vars_cognitive_map = dict(zip(raw_vars, raw_vars_cognitive))
# #raw responses: replicating know human bias?
# raw_vars_known=[0]*16
# raw_vars_known_map = dict(zip(raw_vars, raw_vars_known))
# #raw responses: preference measure?
# raw_vars_pref=list(chain([0]*7, [1]*7, [1]*2))
# raw_vars_pref_map = dict(zip(raw_vars, raw_vars_pref))
# #raw responses: stimuli dependent?
# raw_vars_stim=[1]*16
# raw_vars_stim_map = dict(zip(raw_vars, raw_vars_stim))

# DVs:
DV_vars = ["percent_correct", "fairness_average", "reg_support"]
DV_vars_min = [0, 1, 1]
# fixed range on 07/25/25:
# DV_vars_max=[6,7,7]
DV_vars_max = [100, 7, 7]
# DVs: domain=social?
DV_vars_social = [0, 1, 1]
DV_vars_social_map = dict(zip(DV_vars, DV_vars_social))
# DVs: domain=cognitive?
DV_vars_cognitive = [1, 0, 0]
DV_vars_cognitive_map = dict(zip(DV_vars, DV_vars_cognitive))
# DVs: replicating know human bias?
DV_vars_known = [0, 0, 0]
DV_vars_known_map = dict(zip(DV_vars, DV_vars_known))
# DVs: preference measure?
DV_vars_pref = [0, 1, 1]
DV_vars_pref_map = dict(zip(DV_vars, DV_vars_pref))
# DVs: stimuli dependent?
DV_vars_stim = [1, 1, 1]
DV_vars_stim_map = dict(zip(DV_vars, DV_vars_stim))
# DVs: knowledge question?
DV_vars_know = [1, 0, 0]
DV_vars_know_map = dict(zip(DV_vars, DV_vars_know))
# DVs: political question?
DV_vars_politics = [0, 0, 1]
DV_vars_politics_map = dict(zip(DV_vars, DV_vars_politics))

# merging key
merge_key = ["TWIN_ID"]

# Merge on TWIN_ID
df = pd.merge(df_human, df_twin, on=merge_key, suffixes=("_human", "_twin"))

# Fix dtypes
# for var in raw_vars + DV_vars:
for var in DV_vars:
    df[f"{var}_human"] = pd.to_numeric(df[f"{var}_human"], errors="coerce")
    df[f"{var}_twin"] = pd.to_numeric(df[f"{var}_twin"], errors="coerce")

# build min/max maps from both raw_vars and DV_vars
min_map = {v: mn for v, mn in zip(DV_vars, DV_vars_min)}
# min_map = {v: mn for v, mn in zip(raw_vars,      raw_vars_min)}
# min_map.update({v: mn for v, mn in zip(DV_vars,   DV_vars_min)})

max_map = {v: mx for v, mx in zip(DV_vars, DV_vars_max)}
# max_map = {v: mx for v, mx in zip(raw_vars,      raw_vars_max)}
# max_map.update({v: mx for v, mx in zip(DV_vars,   DV_vars_max)})

# now add _min and _max columns for every variable in the union
for var in min_map:
    df[f"{var}_min"] = min_map[var]
    df[f"{var}_max"] = max_map[var]

# Compute results
results = []
# for var in raw_vars:
#     col_h = f"{var}_human"
#     col_t = f"{var}_twin"
#     min_col = f"{var}_min"
#     max_col = f"{var}_max"
#     if cond_exists:
#         cols = [col_h, col_t, cond_h, cond_t,min_col,max_col]
#     else:
#         cols = [col_h, col_t,min_col,max_col]
#     pair = (
#     df[cols]
#       .dropna(subset=[col_h, col_t])
#     )
#     min_val = pair[min_col].iloc[0]
#     max_val = pair[max_col].iloc[0]
#     n    = len(pair)
#     if n >= 4:
#         r, _    = pearsonr(pair[col_h], pair[col_t])
#         z_f     = np.arctanh(r)
#         se      = 1 / np.sqrt(n - 3)
#         z_crit  = norm.ppf(0.975)
#         lo_z, hi_z = z_f - z_crit*se, z_f + z_crit*se
#         lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
#         z_score    = z_f / se
#         # Accuracy = mean absolute diff / range
#         if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
#             accuracy = np.nan
#         else:
#             # compute mean absolute difference
#             abs_diff      = np.abs(pair[col_h] - pair[col_t])
#             mean_abs_diff = abs_diff.mean()
#             accuracy      = 1 - mean_abs_diff / (max_val - min_val)

#         mean_h = pair[col_h].mean()
#         mean_t = pair[col_t].mean()

#         # Paired t‐test
#         t_stat, p_val = ttest_rel(pair[col_h], pair[col_t])

#         std_h = pair[col_h].std(ddof=1)
#         std_t = pair[col_t].std(ddof=1)

#          # F‐test for equal variances
#         df1 = df2 = n - 1
#         f_stat = (std_h**2 / std_t**2) if std_t>0 else np.nan

#         # two‐tailed p‐value:
#         if not np.isnan(f_stat):
#             p_f = 2 * min(f.cdf(f_stat, df1, df2),
#                           1 - f.cdf(f_stat, df1, df2))
#         else:
#             p_f = np.nan

#         # Effect sizes (Cohen's d) across conditions
#         #    For humans:
#         if cond_exists and len(pair)>3:
#             levels_h = pair[cond_h].unique()
#             if len(levels_h) == 2:
#                 g1 = pair.loc[pair[cond_h]==levels_h[0], col_h]
#                 g2 = pair.loc[pair[cond_h]==levels_h[1], col_h]
#                 n1, n2 = len(g1), len(g2)
#                 # pooled sd
#                 s_pool = np.sqrt(((n1-1)*g1.var(ddof=1)+(n2-1)*g2.var(ddof=1)) / (n1+n2-2))
#                 d_human = (g1.mean() - g2.mean()) / s_pool if s_pool>0 else np.nan
#             else:
#                 d_human = np.nan
#         else:
#             d_human = np.nan

#         #    For twins:
#         if cond_exists and len(pair)>3:
#             levels_t = pair[cond_t].unique()
#             if cond_exists and len(levels_t) == 2:
#                 g1 = pair.loc[pair[cond_t]==levels_t[0], col_t]
#                 g2 = pair.loc[pair[cond_t]==levels_t[1], col_t]
#                 n1, n2 = len(g1), len(g2)
#                 s_pool = np.sqrt(((n1-1)*g1.var(ddof=1)+(n2-1)*g2.var(ddof=1)) / (n1+n2-2))
#                 d_twin = (g1.mean() - g2.mean()) / s_pool if s_pool>0 else np.nan
#             else:
#                 d_twin = np.nan
#         else:
#             d_twin = np.nan
#     else:
#         r = lo_r = hi_r = z_score = accuracy = mean_h = mean_t = t_stat = p_val = std_h = std_t = f_stat = p_f = np.nan
#         d_human = d_twin = np.nan


#     results.append({
#         'study name': study_name,
#         'variable name': var,
#         'variable type (raw response/DV)':     'raw',
#         'correlation between the responses from humans vs. their twins':        r,
#         'CI_lower': lo_r,
#         'CI_upper': hi_r,
#         'z-score for correlation between humans vs. their twins':  z_score,
#         'accuracy between humans vs. their twins': accuracy,
#         'mean_human': mean_h,
#         'mean_twin': mean_t,
#         'paired t-test t-stat': t_stat,
#         'paired t-test p-value': p_val,
#         'std_human': std_h,
#         'std_twin': std_t,
#         'variance test F-stat': f_stat,
#         'variance test p-value': p_f,
#         'effect size based on human': d_human,
#         'effect size based on twin': d_twin,
#         'domain=social?':raw_vars_social_map.get(var, np.nan),
#         'domain=cognitive?':raw_vars_cognitive_map.get(var, np.nan),
#         'replicating know human bias?':raw_vars_known_map.get(var, np.nan),
#         'preference measure?':raw_vars_pref_map.get(var, np.nan),
#         'stimuli dependent?':raw_vars_stim_map.get(var, np.nan),
#         'sample size':        n
#     })

for var in DV_vars:
    col_h = f"{var}_human"
    col_t = f"{var}_twin"
    min_col = f"{var}_min"
    max_col = f"{var}_max"
    if cond_exists:
        cols = [col_h, col_t, cond_h, cond_t, min_col, max_col]
    else:
        cols = [col_h, col_t, min_col, max_col]
    pair = df[cols].dropna(subset=[col_h, col_t])
    min_val = pair[min_col].iloc[0]
    max_val = pair[max_col].iloc[0]
    n = len(pair)
    if n >= 4:
        r, _ = pearsonr(pair[col_h], pair[col_t])
        z_f = np.arctanh(r)
        se = 1 / np.sqrt(n - 3)
        z_crit = norm.ppf(0.975)
        lo_z, hi_z = z_f - z_crit * se, z_f + z_crit * se
        lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
        z_score = z_f / se
        # Accuracy = mean absolute diff / range
        if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
            accuracy = np.nan
        else:
            # compute mean absolute difference
            abs_diff = np.abs(pair[col_h] - pair[col_t])
            mean_abs_diff = abs_diff.mean()
            accuracy = 1 - mean_abs_diff / (max_val - min_val)

        mean_h = pair[col_h].mean()
        mean_t = pair[col_t].mean()

        # Paired t‐test
        t_stat, p_val = ttest_rel(pair[col_h], pair[col_t])

        std_h = pair[col_h].std(ddof=1)
        std_t = pair[col_t].std(ddof=1)

        # F‐test for equal variances
        df1 = df2 = n - 1
        f_stat = (std_h**2 / std_t**2) if std_t > 0 else np.nan
        # two‐tailed p‐value:
        if not np.isnan(f_stat):
            p_f = 2 * min(f.cdf(f_stat, df1, df2), 1 - f.cdf(f_stat, df1, df2))
        else:
            p_f = np.nan

        # Effect sizes (Cohen's d) across conditions
        #    For humans:
        if cond_exists and len(pair) > 3:
            levels_h = pair[cond_h].unique()
            if len(levels_h) == 2:
                g1 = pair.loc[pair[cond_h] == levels_h[0], col_h]
                g2 = pair.loc[pair[cond_h] == levels_h[1], col_h]
                n1, n2 = len(g1), len(g2)
                # pooled sd
                s_pool = np.sqrt(
                    ((n1 - 1) * g1.var(ddof=1) + (n2 - 1) * g2.var(ddof=1)) / (n1 + n2 - 2)
                )
                d_human = (g1.mean() - g2.mean()) / s_pool if s_pool > 0 else np.nan
            else:
                d_human = np.nan
        else:
            d_human = np.nan

        #    For twins:
        if cond_exists and len(pair) > 3:
            levels_t = pair[cond_t].unique()
            if cond_exists and len(levels_t) == 2:
                g1 = pair.loc[pair[cond_t] == levels_t[0], col_t]
                g2 = pair.loc[pair[cond_t] == levels_t[1], col_t]
                n1, n2 = len(g1), len(g2)
                s_pool = np.sqrt(
                    ((n1 - 1) * g1.var(ddof=1) + (n2 - 1) * g2.var(ddof=1)) / (n1 + n2 - 2)
                )
                d_twin = (g1.mean() - g2.mean()) / s_pool if s_pool > 0 else np.nan
            else:
                d_twin = np.nan
        else:
            d_twin = np.nan
    else:
        r = lo_r = hi_r = z_score = accuracy = mean_h = mean_t = t_stat = p_val = std_h = std_t = (
            f_stat
        ) = p_f = np.nan
        d_human = d_twin = np.nan

    results.append(
        {
            "study name": study_name,
            "persona specification": specification_name,
            "variable name": var,
            #        'variable type (raw response/DV)':     'DV',
            "correlation between the responses from humans vs. their twins": r,
            "CI_lower": lo_r,
            "CI_upper": hi_r,
            "z-score for correlation between humans vs. their twins": z_score,
            "accuracy between humans vs. their twins": accuracy,
            "mean_human": mean_h,
            "mean_twin": mean_t,
            "paired t-test t-stat": t_stat,
            "paired t-test p-value": p_val,
            "std_human": std_h,
            "std_twin": std_t,
            "variance test F-stat": f_stat,
            "variance test p-value": p_f,
            "effect size based on human": d_human,
            "effect size based on twin": d_twin,
            "domain=social?": DV_vars_social_map.get(var, np.nan),
            "domain=cognitive?": DV_vars_cognitive_map.get(var, np.nan),
            "replicating know human bias?": DV_vars_known_map.get(var, np.nan),
            "preference measure?": DV_vars_pref_map.get(var, np.nan),
            "stimuli dependent?": DV_vars_stim_map.get(var, np.nan),
            "knowledge question?": DV_vars_know_map.get(var, np.nan),
            "political question?": DV_vars_politics_map.get(var, np.nan),
            "sample size": n,
        }
    )

# results DataFrame
corr_df = pd.DataFrame(results)
print(corr_df)

# save output as csv - unit of observation is comparison between humans and twins:
out_file = f"{study_name} {specification_name} meta analysis.csv"
corr_df.to_csv(out_file, index=False)


#####participant-level data:
def make_long(df, respondent_type):
    # pick off TWIN_ID + the DVs, then melt
    long = df[["TWIN_ID"] + DV_vars].melt(
        id_vars="TWIN_ID", value_vars=DV_vars, var_name="variable_name", value_name="value"
    )

    long["respondent_type"] = respondent_type
    long["study_name"] = study_name
    long["specification_name"] = specification_name
    return long


#########################unique to this study:
# turn the index back into a column so melt can see it
df_human.reset_index(inplace=True)
df_twin.reset_index(inplace=True)
###################

# build the two halves
long_h = make_long(df_human, "human")
long_t = make_long(df_twin, "twin")

# stack them
df_long = pd.concat([long_h, long_t], ignore_index=True)

print(df_long.head())
# save output as csv - unit of observation is TWIN_ID:
out_file = f"{study_name} {specification_name} meta analysis individual level.csv"
df_long.to_csv(out_file, index=False)

print("done")

C:\Users\ot2107\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


  study name persona specification     variable name  \
0  Junk Fees       default persona   percent_correct   
1  Junk Fees       default persona  fairness_average   
2  Junk Fees       default persona       reg_support   

   correlation between the responses from humans vs. their twins  CI_lower  \
0                                          -0.012276             -0.110195   
1                                           0.286766              0.194174   
2                                           0.478548              0.399229   

   CI_upper  z-score for correlation between humans vs. their twins  \
0  0.085880                                          -0.244600        
1  0.374293                                           5.878610        
2  0.550757                                          10.382837        

   accuracy between humans vs. their twins  mean_human  mean_twin  ...  \
0                                 0.518750   51.750000  99.875000  ...   
1                            

In [58]:
# replicate pre-registered analysis

###################################correct answers:
average_total_correct = df_human["percent_correct"].mean()
print(f"Average percent_correct humans: {average_total_correct}")
std_total_correct = df_human["percent_correct"].std()
print(f"Standard deviation of percent_correct humans: {std_total_correct}")

average_total_correct = df_twin["percent_correct"].mean()
print(f"Average percent_correct twis: {average_total_correct}")
std_total_correct = df_twin["percent_correct"].std()
print(f"Standard deviation of percent_correct twins: {std_total_correct}")

acc_h = (df_human["percent_correct"]).dropna()
acc_t = (df_twin["percent_correct"]).dropna()
# align on TWIN_ID if you have them as index, otherwise just drop mismatches:
paired_acc = pd.concat([acc_h, acc_t], axis=1, join="inner").dropna()
n_acc = len(paired_acc)
t_acc, p_acc = ttest_rel(paired_acc.iloc[:, 0], paired_acc.iloc[:, 1])
df_acc = n_acc - 1
print(f"Paired t-test on accuracy: t({df_acc}) = {t_acc:.3f}, p = {p_acc:.3f}  (n={n_acc})")
##############################


################################fairness:
average_fairness_average = df_human["fairness_average"].mean()
print(f"Average fairness_average humans: {average_fairness_average}")
std_fairness = df_human["fairness_average"].std()
print(f"Standard deviation of fairness_average humans: {std_fairness}")

average_fairness_average = df_twin["fairness_average"].mean()
print(f"Average fairness_average twins: {average_fairness_average}")
std_fairness = df_twin["fairness_average"].std()
print(f"Standard deviation of fairness_average twins: {std_fairness}")

fair_h = df_human["fairness_average"].dropna()
fair_t = df_twin["fairness_average"].dropna()
paired_fair = pd.concat([fair_h, fair_t], axis=1, join="inner").dropna()
n_fair = len(paired_fair)
t_fair, p_fair = ttest_rel(paired_fair.iloc[:, 0], paired_fair.iloc[:, 1])
df_fair = n_fair - 1
print(f"Paired t-test on fairness: t({df_fair}) = {t_fair:.3f}, p = {p_fair:.3f}  (n={n_fair})")
#####################################################

Average percent_correct humans: 51.75
Standard deviation of percent_correct humans: 20.869267255691412
Average percent_correct twis: 99.875
Standard deviation of percent_correct twins: 1.8615209838073572
Paired t-test on accuracy: t(399) = -45.888, p = 0.000  (n=400)
Average fairness_average humans: 3.280416666666667
Standard deviation of fairness_average humans: 1.239904301761693
Average fairness_average twins: 3.2716666666666665
Standard deviation of fairness_average twins: 0.6247940345671873
Paired t-test on fairness: t(399) = 0.144, p = 0.886  (n=400)


In [56]:
# regression on political views:

import pandas as pd
import statsmodels.formula.api as smf

df_wave1 = pd.read_csv("Data_Politics.csv", header=0)
# Reverse‐code Q22
pol_map = {
    "Very liberal": 1,
    "Liberal": 2,
    "Moderate": 3,
    "Conservative": 4,
    "Very conservative": 5,
}
# apply it to create a numeric column
df_wave1["political_view"] = df_wave1["Politics"].map(pol_map)
df_human = df_human.drop(columns="political_view", errors="ignore")
# Merge that back into df_human by TWIN_ID
df_human = df_human.merge(df_wave1[["TWIN_ID", "political_view"]], on="TWIN_ID", how="left")
df_twin = df_twin.drop(columns="political_view", errors="ignore")
# Merge that back into df_human by TWIN_ID
df_twin = df_twin.merge(df_wave1[["TWIN_ID", "political_view"]], on="TWIN_ID", how="left")

# 1) Tag and stack
df_h = df_human.copy()
df_h["source"] = "Human"
df_t = df_twin.copy()
df_t["source"] = "AI"

df_all = pd.concat([df_h, df_t], axis=0, ignore_index=True)

df_all["source"] = pd.Categorical(df_all["source"], categories=["Human", "AI"])

# 2) Mean‐center ideology
df_all["ideo_c"] = df_all["political_view"] - df_all["political_view"].mean()

# 3) Fit the regression:
#    reg_support ~ ideo_c + source + ideo_c:source
#    statsmodels will dummy‐code `source` for you
model = smf.ols("reg_support ~ ideo_c * source", data=df_all).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            reg_support   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.320
Method:                 Least Squares   F-statistic:                     126.2
Date:                Mon, 14 Jul 2025   Prob (F-statistic):           6.94e-67
Time:                        15:49:21   Log-Likelihood:                -1349.9
No. Observations:                 800   AIC:                             2708.
Df Residuals:                     796   BIC:                             2726.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               5.0275    